In [1]:
"""
author = Subash Palvel

"""

[nltk_data] Downloading package punkt to C:\Users\Subash
[nltk_data]     Palvel\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [ ]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json

In [2]:
#load the intents.json file from your local device
from google.colab import files
files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Mayank.',
    'Thanks for asking. I am designed by Mayank Bajaj.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Worl

In [5]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
52 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [7]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\Subash Palvel\AppData\Local\Temp\ipykernel_1996\1242873455.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=8, verbose=1)
model.save("model.pkl")

Epoch 1/1000
4/4 [==============================] - 1s 7ms/step - loss: 2.2003 - accuracy: 0.0370
Epoch 2/1000
4/4 [==============================] - 0s 6ms/step - loss: 2.1715 - accuracy: 0.0370
Epoch 3/1000
4/4 [==============================] - 0s 6ms/step - loss: 2.1507 - accuracy: 0.0370
Epoch 4/1000
4/4 [==============================] - 0s 6ms/step - loss: 2.1278 - accuracy: 0.0370
Epoch 5/1000
4/4 [==============================] - 0s 13ms/step - loss: 2.1084 - accuracy: 0.0741
Epoch 6/1000
4/4 [==============================] - 0s 24ms/step - loss: 2.0867 - accuracy: 0.1111
Epoch 7/1000
4/4 [==============================] - 0s 22ms/step - loss: 2.0682 - accuracy: 0.1481
Epoch 8/1000
4/4 [==============================] - 0s 24ms/step - loss: 2.0498 - accuracy: 0.1481
Epoch 9/1000
4/4 [==============================] - 0s 16ms/step - loss: 2.0306 - accuracy: 0.1481
Epoch 10/1000
4/4 [==============================] - 0s 17ms/step - loss: 2.0126 - accuracy: 0.1481
Epoch 11/1000

4/4 [==============================] - 0s 10ms/step - loss: 0.6559 - accuracy: 0.9259
Epoch 84/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.6412 - accuracy: 0.9259
Epoch 85/1000
4/4 [==============================] - 0s 10ms/step - loss: 0.6274 - accuracy: 0.9259
Epoch 86/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.6116 - accuracy: 0.9259
Epoch 87/1000
4/4 [==============================] - 0s 9ms/step - loss: 0.5982 - accuracy: 0.9259
Epoch 88/1000
4/4 [==============================] - 0s 13ms/step - loss: 0.5830 - accuracy: 0.9259
Epoch 89/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.5702 - accuracy: 0.9630
Epoch 90/1000
4/4 [==============================] - 0s 9ms/step - loss: 0.5568 - accuracy: 0.9630
Epoch 91/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.5431 - accuracy: 0.9630
Epoch 92/1000
4/4 [==============================] - 0s 14ms/step - loss: 0.5301 - accuracy: 1.0000
Epoch 93/1000
4/

4/4 [==============================] - 0s 15ms/step - loss: 0.0256 - accuracy: 1.0000
Epoch 246/1000
4/4 [==============================] - 0s 23ms/step - loss: 0.0253 - accuracy: 1.0000
Epoch 247/1000
4/4 [==============================] - 0s 22ms/step - loss: 0.0250 - accuracy: 1.0000
Epoch 248/1000
4/4 [==============================] - 0s 12ms/step - loss: 0.0247 - accuracy: 1.0000
Epoch 249/1000
4/4 [==============================] - 0s 10ms/step - loss: 0.0243 - accuracy: 1.0000
Epoch 250/1000
4/4 [==============================] - 0s 51ms/step - loss: 0.0240 - accuracy: 1.0000
Epoch 251/1000
4/4 [==============================] - 0s 29ms/step - loss: 0.0238 - accuracy: 1.0000
Epoch 252/1000
4/4 [==============================] - 0s 22ms/step - loss: 0.0234 - accuracy: 1.0000
Epoch 253/1000
4/4 [==============================] - 0s 24ms/step - loss: 0.0232 - accuracy: 1.0000
Epoch 254/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.0229 - accuracy: 1.0000
Epoch

4/4 [==============================] - 0s 15ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 408/1000
4/4 [==============================] - 0s 28ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 409/1000
4/4 [==============================] - 0s 31ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 410/1000
4/4 [==============================] - 0s 32ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 411/1000
4/4 [==============================] - 0s 37ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 412/1000
4/4 [==============================] - 0s 35ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 413/1000
4/4 [==============================] - 0s 24ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 414/1000
4/4 [==============================] - 0s 24ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 415/1000
4/4 [==============================] - 0s 24ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 416/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch

4/4 [==============================] - 0s 50ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 570/1000
4/4 [==============================] - 0s 25ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 571/1000
4/4 [==============================] - 0s 43ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 572/1000
4/4 [==============================] - 0s 30ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 573/1000
4/4 [==============================] - 0s 29ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 574/1000
4/4 [==============================] - 0s 23ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 575/1000
4/4 [==============================] - 0s 27ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 576/1000
4/4 [==============================] - 0s 22ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 577/1000
4/4 [==============================] - 0s 16ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 578/1000
4/4 [==============================] - 0s 15ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch

In [ ]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [ ]:
from keras.models import load_model
model = load_model("model.pkl")

In [ ]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [ ]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [ ]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
response('Where are you located?')

Thans for your Interest. I live in India.


In [ ]:
response('That is helpful')

My pleasure


In [ ]:
response('Bye')

Have a nice day


In [ ]:
response("Who are you?")

Thanks for asking. I am Mayank Bajaj, coder by profession but ML enthusiast by passion.
